In [1]:
!pip install  pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 25.8 MB/s eta 0:00:00


In [2]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [3]:
model = pp.LpProblem(name='vitamin-problem',
                     sense=pp.LpMinimize) #use LpMinimize!

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [4]:
# how much sv?
SV = pp.LpVariable(name="SV",
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # when would 'Integer' be needed?

# how much nh?
NH = pp.LpVariable(name="NH",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients_:

In [5]:
SVCoeff=0.2
NHCoeff=0.3
obj_func = SVCoeff*SV + NHCoeff*NH

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [6]:
# SUBJECT TO:
C1= pp.LpConstraint(name='vitaminC',
                    e= 20*SV + 30*NH,
                    rhs=60,
                    sense=pp.LpConstraintGE)
C2= pp.LpConstraint(name='calcium',
                    e= 500*SV + 250*NH,
                    rhs=1000,
                    sense=pp.LpConstraintGE)
C3= pp.LpConstraint(name='iron',
                    e= 9*SV + 2*NH,
                    rhs=18,
                    sense=pp.LpConstraintGE)
C4= pp.LpConstraint(name='niacin',
                    e= 2*SV + 10*NH,
                    rhs=20,
                    sense=pp.LpConstraintGE)
C5= pp.LpConstraint(name='magnesium',
                    e= 60*SV + 90*NH,
                    rhs=360,
                    sense=pp.LpConstraintGE)

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [7]:
model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

6. **Solve the MODEL**: Notice we are not using the _default solver_:

In [8]:
model.solve();

7. **Basic Report**:

From above, you can print:

In [9]:
"Model Status",pp.LpStatus[model.status]

('Model Status', 'Optimal')

Also, get these key results:

In [10]:
Results={"Optimal Solution to maximize revenue":pp.value(model.objective)}
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Optimal Solution to maximize revenue': 1.2, 'NH': 3.1304348, 'SV': 1.3043478}

In [11]:
import pandas as pd
pd.DataFrame.from_dict(Results,orient='index',columns=['info']).map('{:,.2f}'.format)

,info
Optimal Solution to maximize revenue,1.20
NH,3.13
SV,1.30


From above, you know how many of each tablet (SV and NH) to buy each day to meet the minimum requirments while spending the least amount of money.

7. **The Sensitivity of the Result** can be obtained like this:

In [12]:
sensitivityValues = [{'constraints':name, 'shadow price':c.pi, 'slack': c.slack}
                     for name, c in model.constraints.items()]

pd.DataFrame(sensitivityValues).set_index('constraints').map('{:,.2f}'.format)

,shadow price,slack
constraints,,
vitaminC,0.00,-60.00
calcium,0.00,-434.78
iron,0.00,-0.00
niacin,0.00,-13.91
magnesium,0.00,-0.00
